# Lectura de archivos

In [1]:
%matplotlib inline
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
id='1RUtolRcQlR3RGULttM4ZoQaK_Ouow4gc'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('train_labels.csv')
train_labels = pd.read_csv('train_labels.csv', encoding='latin-1', dtype={'building_id': 'int64', 'damage_grade': 'int64'})

In [4]:
id='1b1PsxNDyiTkPNxAI-KDrN_wnoEc5XH2e'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('train_values_selected_features.csv')
train_values = pd.read_csv('train_values_selected_features.csv')

In [5]:
id='1dyLAYyxjQp8JlKDci1o3WS0_TzgBw7lz'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('test_values_selected_features.csv')
test_values = pd.read_csv('test_values_selected_features.csv')

In [6]:
train_values.set_index('building_id', inplace=True)
train_labels.set_index('building_id', inplace=True)
test_values.set_index('building_id', inplace=True)

In [7]:
print(train_values.shape)
print(train_labels.shape)
print(test_values.shape)

(260601, 30)
(260601, 1)
(86868, 30)


# KNN

Jugar un poquito con los hiper-parámetros o meter un grid search o algo.

También podemos poner como h-p el weight='distance' que le asigna menor peso a los que están a distancia más lejana

Más en: https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

In [8]:
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing

Escalamos los datos, importantísimo para KNN (tanto los de train como los de test, sino no tiene sentido)

In [9]:
min_max_scaler = preprocessing.MinMaxScaler()
x_train_scaled = min_max_scaler.fit_transform(train_values)
x_test_scaled = min_max_scaler.fit_transform(test_values)

In [10]:
k_neighboors = KNeighborsClassifier(n_neighbors=10)
k_neighboors.fit(x_train_scaled, train_labels)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                     weights='uniform')

In [11]:
predictions = k_neighboors.predict(x_test_scaled)
predictions

array([3, 2, 2, ..., 2, 3, 2])

#Submission

In [12]:
id = '1NAgGKhUi6tYqhe2yZ9HV9AvzbX-3sJ_B'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('submission_format.csv')
submission_format = pd.read_csv('submission_format.csv', index_col='building_id')

In [13]:
my_submission = pd.DataFrame(data=predictions,
                             columns=submission_format.columns,
                             index=submission_format.index)
my_submission.head()

,damage_grade
building_id,
300051,3
99355,2
890251,2
745817,2
421793,3


In [14]:
my_submission.to_csv('knn_submission.csv')

#Conclusión

No es costoso pero no arroja buenos resultados. Para eliminar outliers puede estar muy bueno.